# Data Engineering

## Reading tabluar data

We start importing the pandas library, and set the number of maximally displayed rows to 5:

In [1]:
import pandas as pd
pd.options.display.max_rows = 5

### Data with delimiters

First, we will import some data on flowers from the provided data sets. Let's have a look at the file:

In [2]:
!head data/text1.csv

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


The first row contains the header (column names), and the columns are separated by a colon. The first 4 columns are floating point numbers, the last column is a string.

The pandas read method `read_csv()` tries to guess all these settings:

In [3]:
pd.read_csv('data/text1.csv')

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


If some guess goes wrong, the parameters can be specified explicitly:

In [4]:
pd.read_csv('data/text1.csv', sep=',', header=0,
            dtype={'Sepal.Length': float, 'Species': str})

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
...,...,...,...,...,...
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


### Fixed with data

In [5]:
!cat data/text2.txt

Anton Maier  Höchstädtplatz 6, 1200 Wien        0699 12345674  
Hugo H. Wolf An den langen Lüssen 47; 1190 Wien +43 4545 454545


In this case, we need to specify the column widths, or start/end of the columns.

In [6]:
data = pd.read_fwf('data/text2.txt', 
                   widths=[13, 35, 15],
                   names=['Name', 'Address', 'Telephone'], 
                   encoding = 'UTF-8')

data

,Name,Address,Telephone
0,Anton Maier,"Höchstädtplatz 6, 1200 Wien",0699 12345674
1,Hugo H. Wolf,An den langen Lüssen 47; 1190 Wien,+43 4545 454545


### Seperating / Joining columns

In the example above, we need some further cleaning: the address should be separated into Street, ZIP code and City. We will try to split the street from the rest using the colon, and then separate ZIP and city with the space.

First, we replace the semi-colon with a colon:

In [7]:
data.Address = data.Address.str.replace(';', ',')

data

,Name,Address,Telephone
0,Anton Maier,"Höchstädtplatz 6, 1200 Wien",0699 12345674
1,Hugo H. Wolf,"An den langen Lüssen 47, 1190 Wien",+43 4545 454545


Now, we extract the address strings as a list, and use the `partition()` method to separate the data:

In [8]:
parts = data.Address.str.partition(', ')

parts

,0,1,2
0,Höchstädtplatz 6,",",1200 Wien
1,An den langen Lüssen 47,",",1190 Wien


This results in a data frame with the three columns. We add the first column (address) as a new `Street` column:

In [9]:
data['Street'] = parts[0]

data

,Name,Address,Telephone,Street
0,Anton Maier,"Höchstädtplatz 6, 1200 Wien",0699 12345674,Höchstädtplatz 6
1,Hugo H. Wolf,"An den langen Lüssen 47, 1190 Wien",+43 4545 454545,An den langen Lüssen 47


Then, we further split the third column...

In [10]:
tmp = parts[2].str.partition(' ')

tmp

,0,1,2
0,1200,,Wien
1,1190,,Wien


... and add the result as `ZIP` and `City` columns. Finally, we delete the old `Address` column:

In [11]:
data['ZIP'] = tmp[0]
data['City'] = tmp[2]
data.drop('Address', axis=1, inplace=True)

data

,Name,Telephone,Street,ZIP,City
0,Anton Maier,0699 12345674,Höchstädtplatz 6,1200,Wien
1,Hugo H. Wolf,+43 4545 454545,An den langen Lüssen 47,1190,Wien


Recombining columns is easy, using the `+` operator:

In [12]:
data['Address'] = data.ZIP + ' ' + data.City + ', ' + data.Street
data.drop(['ZIP', 'City', 'Street'], axis=1)

,Name,Telephone,Address
0,Anton Maier,0699 12345674,"1200 Wien, Höchstädtplatz 6"
1,Hugo H. Wolf,+43 4545 454545,"1190 Wien, An den langen Lüssen 47"


### Wide and Long format

In [13]:
us_arrests = pd.read_csv('data/USArrests.csv')

us_arrests

,State,Murder,Assault,Rape
0,Alabama,13.2,236,21.2
1,Alaska,10.0,263,44.5
...,...,...,...,...
48,Wisconsin,2.6,53,10.8
49,Wyoming,6.8,161,15.6


The columns `Murder`, `Assault` and `Rape` are three different types of crime. A cleaner structure would be a string column `Crime`, along with a numeric column `Arrests`:

In [14]:
us_long = us_arrests.melt(id_vars=['State'], var_name='Crime', value_name='Arrests')

us_long

,State,Crime,Arrests
0,Alabama,Murder,13.2
1,Alaska,Murder,10.0
...,...,...,...
148,Wisconsin,Rape,10.8
149,Wyoming,Rape,15.6


This representation is called *long* format. To transform it back to the initial *wide* format, we use the `pivot()` method:

In [15]:
us_long.pivot(index = 'State', columns = 'Crime', values = 'Arrests')

Crime,Assault,Murder,Rape
State,,,
Alabama,236.0,13.2,21.2
Alaska,263.0,10.0,44.5
...,...,...,...
Wisconsin,53.0,2.6,10.8
Wyoming,161.0,6.8,15.6


### Missing data

In [16]:
!cat data/text3.txt

Class Sex    Age    Died  Survived         
1st   Male   Child  0      5
             Adult  118    57
      Female Child  0      1 
             Adult  4      140
2nd   Male   Child  0      11
             Adult  154    14
      Female Child  0      13
             Adult  13     80
3rd   Male   Child  35     13
             Adult  387    75
      Female Child  17     14
             Adult  89     76
Crew  Male   Child  0      ??
             Adult  670    192
      Female Child  0      ??
             Adult  3      20


The Class and Sex labels have not been repeated in each row, and there are two missing values (`??`) for Children in the Crew Class. When we read in the data using `read_fwf()`, we get:

In [17]:
data = pd.read_fwf('data/text3.txt', na_values='??')

data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,NaN,NaN,Adult,118,57.0
...,...,...,...,...,...
14,NaN,Female,Child,0,NaN
15,NaN,NaN,Adult,3,20.0


Both empty cells and missing values are represented by the `NaN` symbol. To clean the data, we need to replace `NaN` in the `Class` and `Sex` columns with the last known label from the top, and to handle the "real" missing values suitably. We start handling the missing data in the `Class` and `Sex` columns using the `fillna()` method:

In [18]:
data[['Class', 'Sex']] = data[['Class', 'Sex']].fillna(method='ffill')

data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
14,Crew,Female,Child,0,NaN
15,Crew,Female,Adult,3,20.0


The remaining `NaN` values could either be removed, or replaced by some sensible value. If the values were missing "completely at random", we could, e.g., use the mean of the other values. In this case, however, these values are not missing at random: Crew members clearly were all adult, so the correct replacement value is 0.

To filter all rows with missing values, we can use the `isna()` method, indicating, for each cell, whether the value is missing or not. Applying `any()` to the *transposed* table will give us, for each row, whether at least one value is missing:

In [19]:
data.isna().T.any()

0     False
1     False
      ...  
14     True
15    False
Length: 16, dtype: bool

Thus, all incomplete rows are given by:

In [20]:
data[data.isna().T.any()]

,Class,Sex,Age,Died,Survived
12,Crew,Male,Child,0,NaN
14,Crew,Female,Child,0,NaN


These could be removed using `dropna()`:

In [21]:
data.dropna()

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
13,Crew,Male,Adult,670,192.0
15,Crew,Female,Adult,3,20.0


... or replaced with 0 again using `fillna()`:

In [22]:
data.fillna(0, inplace = True)

data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5.0
1,1st,Male,Adult,118,57.0
...,...,...,...,...,...
14,Crew,Female,Child,0,0.0
15,Crew,Female,Adult,3,20.0


__Final note:__ because `NaN` is only available for floating point values, the data type of `Survived` is incorrect. To fix this, we can use:

In [23]:
data.Survived = data.Survived.astype(int)

data

,Class,Sex,Age,Died,Survived
0,1st,Male,Child,0,5
1,1st,Male,Adult,118,57
...,...,...,...,...,...
14,Crew,Female,Child,0,0
15,Crew,Female,Adult,3,20


## Data Manipulation

We first import `numpy` and `pandas` libraries:
* Pandas
* NumPyPy

In [24]:
import pandas as pd
import numpy as np

Next, we import the iris data set as a pandas data frame, and show the first few rows (the "head"):

In [25]:
iris = pd.read_csv('data/iris.csv')

iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### Selecting columns and rows

Columns can be selected by indexing the data frame with a *list* of column names:

In [26]:
iris[ ['Sepal.Length', 'Sepal.Width'] ].head()

,Sepal.Length,Sepal.Width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


Columns can be dropped using the `drop()` method, applied to axis 1 (= columns):

In [27]:
iris.drop('Species', axis = 1).head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


More complex filtering can be done by extracting the columns string names from the data frame, and then applying some filtering method like `startswith()` or `findall()`. The resulting indexing object (a pandas `Series`) can then be used with the `loc()` method to extract the corresponding columns.

In [28]:
iris.loc[:,iris.columns.str.startswith('Sepal')].head()

,Sepal.Length,Sepal.Width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


In [29]:
iris.loc[:,iris.columns.str.contains('^S.*\\.')].head()

,Sepal.Length,Sepal.Width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


Rows can simply be selected by specifying one or more predicates within the indexing operator:

In [30]:
iris[iris.Species == 'versicolor'].head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor


In [31]:
iris[(iris['Sepal.Length'] > 5) & (iris['Sepal.Width'] > 4)]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
15,5.7,4.4,1.5,0.4,setosa
32,5.2,4.1,1.5,0.1,setosa
33,5.5,4.2,1.4,0.2,setosa


### Transforming variables

Change content (modify existing variable):

In [32]:
iris['Sepal.Length'] = iris['Sepal.Length'].round()

iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.0,3.5,1.4,0.2,setosa
1,5.0,3.0,1.4,0.2,setosa
2,5.0,3.2,1.3,0.2,setosa
3,5.0,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Adding a new variable, based on an existing one, can be done by using the `where()` funtion from the `numpy` library. It works like vectorized ternary operator:

In [33]:
iris['Sepal'] = np.where(iris['Sepal.Length'] > 5, 'Long', 'Short')

iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Sepal
0,5.0,3.5,1.4,0.2,setosa,Short
1,5.0,3.0,1.4,0.2,setosa,Short
2,5.0,3.2,1.3,0.2,setosa,Short
3,5.0,3.1,1.5,0.2,setosa,Short
4,5.0,3.6,1.4,0.2,setosa,Short


### Renaming variables

Renaming variables can be done using the `rename()` method. The simplest way is by specifying a dictionary of old name/new name pairs as the `columns` argument:

In [34]:
iris.rename(columns = {'Sepal.Length': 'Sepal_Length'}).head()

,Sepal_Length,Sepal.Width,Petal.Length,Petal.Width,Species,Sepal
0,5.0,3.5,1.4,0.2,setosa,Short
1,5.0,3.0,1.4,0.2,setosa,Short
2,5.0,3.2,1.3,0.2,setosa,Short
3,5.0,3.1,1.5,0.2,setosa,Short
4,5.0,3.6,1.4,0.2,setosa,Short


### Sorting and summarizing

The `sort_values()` method allows sorting according to several columns - descending or ascending:

In [35]:
iris.sort_values(by=['Species', 'Sepal.Length'], ascending=[False, True])

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Sepal
106,5.0,2.5,4.5,1.7,virginica,Short
100,6.0,3.3,6.0,2.5,virginica,Long
...,...,...,...,...,...,...
33,6.0,4.2,1.4,0.2,setosa,Long
36,6.0,3.5,1.3,0.2,setosa,Long


The simplest way to compute summary statistics is to use the `describe()` method. It computes, for each variable, count, mean, standard deviation, and all quartiles (including min/max).

In [36]:
iris.describe()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
count,150.00,150.000000,150.000,150.000000
mean,5.86,3.057333,3.758,1.199333
...,...,...,...,...
75%,6.00,3.300000,5.100,1.800000
max,8.00,4.400000,6.900,2.500000


These statistics can also computed individually by first selecting a variable, and then calling the corresponding method:

In [37]:
iris['Sepal.Length'].mean()

5.86

In [38]:
iris.Species.value_counts()

setosa        50
versicolor    50
virginica     50
Name: Species, dtype: int64

In [39]:
iris.mean(numeric_only=True)

Sepal.Length    5.860000
Sepal.Width     3.057333
Petal.Length    3.758000
Petal.Width     1.199333
dtype: float64

### Grouping

An important feature is grouping. After grouping, all summary methods are applied to each group separately. In the following, we first group by the `Species` column, and then apply some summary methods:

In [40]:
iris.groupby('Species').mean(numeric_only=True)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.00,3.428,1.462,0.246
versicolor,6.04,2.770,4.260,1.326
virginica,6.54,2.974,5.552,2.026


In [41]:
iris.groupby('Species')['Sepal.Length'].describe()

,count,mean,std,min,25%,50%,75%,max
Species,,,,,,,,
setosa,50.0,5.00,0.451754,4.0,5.0,5.0,5.0,6.0
versicolor,50.0,6.04,0.532993,5.0,6.0,6.0,6.0,7.0
virginica,50.0,6.54,0.734291,5.0,6.0,6.0,7.0,8.0
